In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from commit_transformer.tokenizer import build_vocab
from commit_transformer.preprosser import CommitDataset
from commit_transformer.model import CombinedModel

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df


/tmp/ipykernel_15719/4142705696.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,label,message,diff
0,0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...
2,1,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...
3,1,Release 2.72.4+171110,diff --git a/application/config/version.php b/...
4,0,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...
10107,0,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....
10108,1,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,0,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,0,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [3]:
df = df[['message','diff','label']] 
df

,message,diff,label
0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...,0
1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...,1
2,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...,1
3,Release 2.72.4+171110,diff --git a/application/config/version.php b/...,1
4,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...,0
...,...,...,...
10107,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....,0
10108,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...,1
10109,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...,0
10110,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...,0


In [4]:
data = []
for index, row in df.iterrows():
  data.append([row['message'],row['diff'],row['label']])

In [6]:

# Split the dataset
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Build vocabulary
vocab = build_vocab(train_data)

# Create Datasets and DataLoaders
train_dataset = CommitDataset(train_data, vocab)
val_dataset = CommitDataset(val_data, vocab)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Initialize the model
vocab_size = len(vocab)
embed_dim = 128  # Dimension for embeddings
num_heads = 8
hidden_dim = 512
num_layers = 6
dropout = 0.3

model = CombinedModel(vocab_size, embed_dim, num_heads, hidden_dim, num_layers, dropout)

# Train the model
model.trainer(train_loader, val_loader, num_epochs=10)

/root/miniconda3/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.6512803960694626
Epoch 2/10, Loss: 0.6234879508437846
Epoch 3/10, Loss: 0.6016558967825216
Epoch 4/10, Loss: 0.5726168588422268
Epoch 5/10, Loss: 0.5505436816705546
Epoch 6/10, Loss: 0.5298752844657587
Epoch 7/10, Loss: 0.5075749235426484
Epoch 8/10, Loss: 0.49357474773474364
Epoch 9/10, Loss: 0.4775251802277895
Epoch 10/10, Loss: 0.465182170478016
Validation Accuracy: 0.750370736529906
Precision: 0.6884561891515995
Recall: 0.6378865979381443
F1-Score: 0.6622073578595318
